In [64]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [65]:
# re-size all the images to this
IMAGE_SIZE = [224, 224]
train_path = 'Datasets/train'
valid_path = 'Datasets/test'

In [66]:
# Import the VGG 16 library as show below and add pre-processing layer to the front of VGG
# Here we will be using imagenet weights
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [67]:
# don't train existing weights
for layer in vgg.layers:
    layer.trainable = False

In [68]:
folders = glob('Datasets/train/*')

In [69]:
# our layers - you can add more if you want
x = Flatten()(vgg.output)

In [70]:
prediction = Dense(len(folders), activation='softmax')(x)
# create a model object
model = Model(inputs=vgg.input, outputs=prediction)

In [71]:
# view the structure of the model
model.summary()

Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [72]:
# tell the model what cost and optimization method to use
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [73]:
# Use the image data generator to import the images from the dataset
train_dataGen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_dataGen = ImageDataGenerator(rescale=1./255)

In [74]:
# Make sure you provide the same target size as initialized for the image size
training_set = train_dataGen.flow_from_directory('Datasets/train', target_size=(224,224), batch_size=32, class_mode='categorical')

Found 5216 images belonging to 2 classes.


In [75]:
test_set = test_dataGen.flow_from_directory('Datasets/test', target_size=(224,224), batch_size=32, class_mode='categorical')

Found 624 images belonging to 2 classes.


In [82]:
import sys
from PIL import Image
sys.modules['Image'] = Image
# fit the model
# Run the cell. It will take some time to execute
r = model.fit_generator(training_set, validation_data=test_set, epochs=5, steps_per_epoch=len(training_set), validation_steps=len(test_set))

Epoch 1/5


ImportError: Could not import PIL.Image. The use of `load_img` requires PIL.

In [83]:
# plot the loss
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')
# plot the accuracy
plt.plot(r.history['acc'], label='train acc')
plt.plot(r.history['val_acc'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')

NameError: name 'r' is not defined

In [ ]:
# save it as a h5 file
import tensorflow as tf
model.save('model_vgg16.h5')

In [ ]:
# Similarly you can use the same template for Vgg 19, Resnet50, Mobilenet. All you have to import the library. Below are the examples
from keras.applications.vgg19 import VGG19
from keras.applications.inception_v3 import InceptionV3